#  PCA Portfolio Variance Analysis 

This notebook demonstrates SVD-based Principal Component Analysis (PCA) for portfolio risk analysis and illustrates its relationship to the traditional covariance-matrix formulation of PCA.

## SVD Algorithm 

Recall that SVD decomposes a matrix $A \in \mathbb{R}^{m \times n}$ into the form $A=U \Sigma V^T$, which is computed in the following way: 
- Determine the eigenvalues $\lambda_i$ and eigenvectors $v_i$ of $A^TA$, which always has real, nonnegative eigenvalues since it is square, symmetric, and positive semidefinite. Order them descending $\lambda_1\geq\lambda_2\geq...\geq0$ with coresponding ordered eigenvectors $v_i$. 
- Construct $m \times n$ diagonal matrix $\Sigma = diag(\sigma_1, \sigma_2,...)$ where $\sigma_i = \sqrt{\lambda_i}$ are the singular values.
- Construct  $n \times n$ orthogonal matrix $V= [v_1, v_2,...,v_n]$ with orthonormal eigenvectors $v_i$.
- Construct $m \times m$ orthogonal matrix $U$ by the formula $u_i = \frac{Av_i}{\sigma_i}$ for $\sigma_i \neq 0$. 

This means that $A^TAV = (V \Sigma^TU^T)(U \Sigma V^T)V = V(\Sigma^T \Sigma)\Longleftrightarrow A^TA v_i = \sigma_i^2 v_i$, i.e. $V$ contains the eigenvectors of $A^TA$, and $\Sigma$ contains the roots of the eigenvalues. Likewise, $AA^T u_i = \sigma_i^2 u_i$, so $U$ contains the eigenvectors of $AA^T$. 
Intuitively, the computation $A\vec{x}= U\Sigma V^T\vec{x}$, any generic linear transformation that stretches the unit sphere in $\mathbb{R}^n$  into a scaled ellipsoid in $\mathbb{R}^m$, is equivalent to 
- Rotation into principal directions ($V^T$) 
- A stretch or compression along the principal axes by corresponding singular values ($\Sigma$)
- Rotation back into the output space ($U$)

Note: NumPy may store diagonal matrices as 1D arrays of singular values for memory efficiency, as you may see below. 

In [88]:
import numpy as np

A = np.array([[1,2,3],[4,5,6],[7,8,9]]) 
U, S, VT = np.linalg.svd(A) 

print("U: ", U, "\nSigma: ", S, "\nV^T: ", VT)

U:  [[-0.21483724  0.88723069  0.40824829]
 [-0.52058739  0.24964395 -0.81649658]
 [-0.82633754 -0.38794278  0.40824829]] 
Sigma:  [1.68481034e+01 1.06836951e+00 3.33475287e-16] 
V^T:  [[-0.47967118 -0.57236779 -0.66506441]
 [-0.77669099 -0.07568647  0.62531805]
 [-0.40824829  0.81649658 -0.40824829]]


## Principal Directions, Lagrange Derivation, and Low-Rank Approximation

Rigorously, the SVD determines the direction $\vec{v}$ with $||\vec{v}||=1$ that maximizes the quantity $||A\vec{v}||$. One can prove this with  Lagrange:
- Find max $f(\vec{v}) = ||A\vec{v}||^2=\vec{v}^T(A^TA)\vec{v}$ subject to constraint $g(\vec{v})=||\vec{v}||^2=\vec{v}^T\vec{v}=v_1^2+v_2^2+...=1$. One can view $f(\vec{v})$ as a surface over a unit sphere $g(\vec{v})=1$ in $\mathbb{R}^n$.
- Then we solve for $\nabla f(\vec{v}) || \nabla g(\vec{v}) \Longleftrightarrow \nabla f(\vec{v})= \lambda \nabla g(\vec{v}) $. Note that $\nabla g(\vec{v})= 2\vec{v}$ and $\nabla f( \vec{v} ) = 2A^TA\vec{v}$ implies $A^TA\vec{v} = \lambda \vec{v}$
- This is exactly the problem of solving for eigenvectors and eigenvalues of $A^TA$. Moreover, $||A\vec{v}||^2= \vec{v}^TA^TA\vec{v}= \lambda ||\vec{v}||^2$ takes max value for the largest eigenvalue.

Moreover, since the eigenvectors of different eigenvalues of a symmetric matrix are orthogonal, we maximize $||A\vec{v}|$ in perpendicular directions. Thus, PCA identifies the directions of maximal $||A\vec{v}||$, ordered by value of $\lambda$, in perpendicular directions. This is why it is useful for covariance matrices-- we can see which direction results in greatest variance.

It has been shown that the best lower rank (i.e. lower dimensional) approximation of a matrix $A$ as measured by the Frobenius norm $||A||_F = \sqrt{\sum_i\sum_j|a_{ij}|^2}$ is the SVD. The Forbenius error is given by $||A-A_k||_F^2=\sum_{i=k+1}^{r}\sigma_i^2$ and the amount of variance accounted for by our approximation in $k$ of our $r$ principal directions is given by $\frac{\sum_{i=1}^{k} \sigma_i^2}{\sum_{i=1}^r \sigma_i^2}$. 

Below is an example showing how to retain the two largest singular values of a $3 \times 3$ matrix and compute the approximation error using the Frobenius norm (default in NumPy). 

In [155]:
A = np.array(
[[3.0, 1.0, 2.0],
 [2.5, 0.9, 1.8],
 [2.0, 0.8, 1.6],
 [1.5, 0.4, 1.0],
 [0.5, 0.2, 0.4]]
)


U, S, VT = np.linalg.svd(A)
k = 2
Sigma_k = np.diag(S[:k]) #restrict to k singular values
U_k =  U[:,:k] 
VT_k = VT[:k, :]
A_approx =  U_k @ Sigma_k @ VT_k
fro_error = np.linalg.norm(A - A_approx, 'fro')
variance_acc = 100 * np.sum(S[:k]**2) / np.sum(S**2)

print("U shape:", U_k.shape, "Sigma shape:", Sigma_k.shape, "V shape:", VT_k.shape, "A shape:", A.shape)
print("Singular Values:", S)
print("Approximation error using 2 components:", fro_error,
      " (≈ third singular value).")
print(f"The first 2 principal components account for {variance_acc:.3f}% of total variance.")
print("\nOriginal Matrix A:\n", A)
print("\nReconstructed from top 2 PCs:\n", A_approx)

U shape: (5, 2) Sigma shape: (2, 2) V shape: (2, 3) A shape: (5, 3)
Singular Values: [5.94177114 0.22267995 0.07595627]
Approximation error using 2 components: 0.07595627220785459  (≈ third singular value).
The first 2 principal components account for 99.984% of total variance.

Original Matrix A:
 [[3.  1.  2. ]
 [2.5 0.9 1.8]
 [2.  0.8 1.6]
 [1.5 0.4 1. ]
 [0.5 0.2 0.4]]

Reconstructed from top 2 PCs:
 [[2.99573809 0.97021265 2.0206212 ]
 [2.49889429 0.89227194 1.80534998]
 [2.00205048 0.81433123 1.59007877]
 [1.50740932 0.45178529 0.96415006]
 [0.50051262 0.20358281 0.39751969]]


## SVD, Covariance, and PCA
Earlier we saw how SVD is equivalent to solving for $\vec{v}$ maximizing $$||A\vec{v}||^2=\vec{v}^T(A^TA)\vec{v} \quad \text{ subject to  } \quad ||\vec{v}||=1$$We now relate it to covariance. Suppose $X \in \mathbb{R}^{m \times n}$ is a *centered* data matrix with rows as variables and columns as observations, then the covariance matrix is given by $\Sigma=\frac{1}{n}X^TX$. Then  $$||X\vec{v}||^2=\vec{v}^T(X^TX)\vec{v}=n\vec{v}^T\Sigma\vec{v}.$$ Thus finding $\vec{v}$ maximizing $||X\vec{v}||$ is equivalent to finding $\vec{v}$ maximizing $\vec{v}^T\Sigma\vec{v}$, the direction where projecting onto $\vec{v}$ yields max variance. 

The vectors in order in the columns of $U$ give the principal directions when $X$ has rows = features, columns = observations. If the rows = observations and columns = features, then use the rows of $V^T$. The principal directions take the form $PC = w_1 \text{ stock1} + w_2 \text{ stock2} + w_3 \text{ stock3} + w_4 \text{ stock4} + w_5 \text{ stock5}$, as linear combination of the original features. Projecting (by calculating $X_{proj}=U^TX$; we use $XV^T$ if rows = observations, columns = features) the 5D stock data onto these PCs gives scalar outputs with the greatest spread of projected values along the line — the direction of maximum historical portfolio variance. If we interpret the PCs as profit functions, they represent the riskiest weighting of stocks: the one whose profit varies the most.

![See illustrative GIF in `data/PCAVisual.gif`](../data/PCAVisual.gif)

Intuitively: 
- When all PC1 (first principal component) weights share the same sign, total variance is largest when all stocks rise or fall together along the line/profit function.
- If PC1 weights alternate in sign, movements of stocks together partially cancel some of the variance (i.e., do not move directly along the line of max variance), reducing total variance. This is a mathematical expression of diversification. 
  
Important notes:
- PCA identifies directions of maximum *sample* variance, not the true population variance. Strong principal components can appear for independent variables, reflecting sample scale or random co-fluctuations rather than true correlation. 
- Changing sample size may affect SVD. As the sample size (e.g., days) increases, random effects diminish, and the dominant direction stabilizes if genuine correlations exist.
- This is the SVD-based formulation of PCA. Traditional PCA explicitly computes the covariance matrix and then finds its eigenvalues and eigenvectors. The eigenvalues from the SVD (squares of singular values) are the same as from PCA except scaled by a constant factor, typically 
1/n depending on how covariance is defined. Thus, while the numerical values of the eigenvalues may differ by that scaling constant, the principal component directions are identical.

PCA reveals dominant correlation patterns, aiding dimensionality reduction, risk analysis, and noise filtering—since weaker components often represent residual or random noise.

In [144]:
np.random.seed(42) 
portfolio_returns = np.random.normal(0.05, 0.02, (5, 30))

def svd(A, k):
    A_centered = A - np.mean(A, axis=0) #Make sure you center before PCA, set the axis depending on whether row = features or col = features.
    U, S, VT = np.linalg.svd(A_centered, full_matrices= False) #set full_matrices = false for large matricies 
    x = np.sum(S[:k]**2) / np.sum(S**2)
    print("Here are the singular values:\n  ", S) 
    print("Here are the associated first", k, "principal components in columns of U, in order:\n ", U[:, :k])
    print("VARIANCE EXPLAINED: \n ", x)

svd(portfolio_returns, 3)

Here are the singular values:
   [1.15291338e-01 1.11092306e-01 9.73805149e-02 8.43686795e-02
 5.14097015e-17]
Here are the associated first 3 principal components in columns of U, in order:
  [[ 0.24815575 -0.75162321  0.15124529]
 [-0.3109321  -0.23204021  0.08968654]
 [ 0.50923836  0.24648983 -0.68604702]
 [-0.71453136  0.21246153 -0.22429635]
 [ 0.26806936  0.52471206  0.66941154]]
VARIANCE EXPLAINED: 
  0.8314635698305499
